In [6]:
import numpy as np
from PIL import Image
import os
import pandas as pd

In [7]:
class Make_Images(object):
    def __init__(self, indir = os.getcwd(), outdir = os.getcwd()):
        self.inder = indir
        self.outdir = outdir
        self.saved_images = []
        self.image_dict = {}
        self.sub_dirs = []
        for directory in os.listdir(self.inder):
            cnt_dir = os.path.join(self.inder,directory)
            if os.path.splitext(cnt_dir)[1] == '' and directory[0] != '.':
                self.sub_dirs.append(cnt_dir)

        self.image_dirs = []
        for directory in self.sub_dirs:
            for subdir in os.listdir(directory):
                if subdir[0] == 'f':
                    self.image_dirs.append(os.path.join(self.inder,directory,subdir))

        self.make_out_dir()
        self.build_image_dict()
    def make_out_dir(self):
        """
        Creates the output directory.
        """
        save_path = os.path.join(self.outdir, 'fullset/')
        if not os.path.exists(save_path):
            os.makedirs(save_path)
            print "Creating path: " + save_path
    def build_image_dict(self):
        """
        Pulls an image in with PIL and
        """
        print "Building image dictionary"
        for directory in self.image_dirs:
            images = [x for x in os.listdir(directory) if x[-4:] == '.png']
            for image in images:
                self.image_dict[image] = {}
                self.image_dict[image]['path'] = os.path.join(directory,image)
            image_annot = pd.read_csv(os.path.join(directory,'frameAnnotations.csv'), delimiter=';')
            for row in range(len(image_annot)):
                self.image_dict[image_annot.loc[row][0]]['bounds'] = [float(x) for x in str(image_annot.loc[row][2:6]).split() if x.isdigit()]
                self.image_dict[image_annot.loc[row][0]]['type'] = image_annot.loc[row][1]
    def crop_images(self):
        """
        Crops image
        """
        for im in self.image_dict.items():
            img = Image.open(im[1]['path'])
            img2 = img.crop(tuple(im[1]['bounds']))
            longer_side = max(img2.size)
            horizontal_padding = (longer_side - img2.size[0]) / 2
            vertical_padding = (longer_side - img2.size[1]) / 2
            img3 = img2.crop(
            (
                -horizontal_padding,
                -vertical_padding,
                img2.size[0] + horizontal_padding,
                img2.size[1] + vertical_padding
            )
            )
            if img3.size[0] > img3.size[1]:
                img4 = img3.crop((0,0,img3.size[1],img3.size[1]))
            elif img3.size[0] < img3.size[1]:
                img4 = img3.crop((0,0,img3.size[0],img3.size[0]))
            else:
                img4 = img3
            image_str = '{}{}'.format(im[1]['type'], str(img4.size))
            x = 0
            while image_str + str(x)in self.saved_images:
                x += 1
            image_str += str(x)
            img4.save(os.path.join(self.outdir, "fullset/", "{}.jpg".format(image_str)))
            self.image_dict[im[0]]['cropped'] = image_str
            self.saved_images.append(image_str)
            if len(os.listdir(os.path.join(self.outdir, 'fullset/'))) % 1000 == 0:
                print len(os.listdir(os.path.join(self.outdir, 'fullset/')))


In [8]:
root = '/Users/brian/Documents/datasets/lisa_set'

In [9]:
sub_dirs = []
for directory in os.listdir(root):
    cnt_dir = os.path.join(root,directory)
    if os.path.splitext(cnt_dir)[1] == '' and directory[0] != '.':
        sub_dirs.append(cnt_dir)

In [10]:
image_dirs = []
for directory in sub_dirs:
    for subdir in os.listdir(directory):
        if subdir[0] == 'f':
            image_dirs.append(os.path.join(root,directory,subdir))

In [11]:
image_dirs

['/Users/brian/Documents/datasets/lisa_set/aiua120214-0/frameAnnotations-DataLog02142012_external_camera.avi_annotations',
 '/Users/brian/Documents/datasets/lisa_set/aiua120214-1/frameAnnotations-DataLog02142012_001_external_camera.avi_annotations',
 '/Users/brian/Documents/datasets/lisa_set/aiua120214-2/frameAnnotations-DataLog02142012_002_external_camera.avi_annotations',
 '/Users/brian/Documents/datasets/lisa_set/aiua120306-0/frameAnnotations-DataLog02142012_002_external_camera.avi_annotations',
 '/Users/brian/Documents/datasets/lisa_set/aiua120306-1/frameAnnotations-DataLog02142012_003_external_camera.avi_annotations',
 '/Users/brian/Documents/datasets/lisa_set/vid0/frameAnnotations-vid_cmp2.avi_annotations',
 '/Users/brian/Documents/datasets/lisa_set/vid1/frameAnnotations-vid_cmp1.avi_annotations',
 '/Users/brian/Documents/datasets/lisa_set/vid10/frameAnnotations-MVI_0122.MOV_annotations',
 '/Users/brian/Documents/datasets/lisa_set/vid11/frameAnnotations-MVI_0123.MOV_annotations',

In [12]:
image_dict = {}
for directory in image_dirs:
    images = [x for x in os.listdir(directory) if x[-4:] == '.png']
    for image in images:
        image_dict[image] = {}
        image_dict[image]['path'] = os.path.join(directory,image)
    image_annot = pd.read_csv(os.path.join(directory,'frameAnnotations.csv'), delimiter=';')
    for row in range(len(image_annot)):
        image_dict[image_annot.loc[row][0]]['bounds'] = [float(x) for x in str(image_annot.loc[row][2:6]).split() if x.isdigit()]
        image_dict[image_annot.loc[row][0]]['type'] = image_annot.loc[row][1]

In [14]:
image_dict.items()[0][1]['path']

'/Users/brian/Documents/datasets/lisa_set/aiua120306-1/frameAnnotations-DataLog02142012_003_external_camera.avi_annotations/stopAhead_1333396732.avi_image0.png'

In [13]:
paths = []
images = []
for i in image_dict.items():
    images.append(i[0])
    paths.append(i[1]['path'])

In [17]:
len(np.unique(paths))

6618

In [18]:
len(np.unique(images))

6618

In [19]:
images[0]

'stopAhead_1333396732.avi_image0.png'

In [20]:
paths[0]

'/Users/brian/Documents/datasets/lisa_set/aiua120306-1/frameAnnotations-DataLog02142012_003_external_camera.avi_annotations/stopAhead_1333396732.avi_image0.png'

In [21]:
len(paths)

6618

In [2]:
import json

In [14]:
with open('image_dict_full.json', 'w') as f:
    json.dump(image_dict, f)

In [15]:
with open('image_dict_full.json') as data_file:    
    data = json.load(data_file)

In [16]:
type(data)

dict